In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns

In [ ]:
date_cols = [
    'Auftragseingang', 
    'Auftragsende_SOLL',
    'AFO_Start_SOLL',
    'AFO_Ende_SOLL',
    'AFO_Start_IST',
    'AFO_Ende_IST',
    'Auftragsende_IST' 
]

data = pd.read_csv(
    '../data/processed/data_cleaned_timestamp_1.csv',
    parse_dates=date_cols,
    sep=',',
    low_memory=False
)